In [ ]:
# default_exp core

# Core reading utils
> API details.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
import fnmatch
import socket
from pathlib import Path

import xarray as xr

hostname = socket.gethostname()
if fnmatch.fnmatch(hostname, "luna?.diviner.ucla.edu"):
    ON_LUNA = True
else:
    ON_LUNA = False

In [ ]:
# export
def raster_to_xarray(fpath, chunk_scale=2):
    """Read raster image into an xarray.DataArray.

    Using the `chunks` keyword in the open_rasterio method
    activates the return of an out-of-memory virtual array instead
    of the in-memory xarray.DataArray

    fpath: pathlib.Path, str
    chunk_scale: int
        Multiplier for the hardcoded 2048/1024 chunk-sizes for
        x/y axes.
    """
    fpath = Path(fpath)
    return xr.open_rasterio(
        fpath, chunks={"x": chunk_scale * 2048, "y": chunk_scale * 1024}
    )

Using the `chunks` parameter in `open_rasterio` returns a `dask.array` version of `xarray.DataArray`.
To actually see values of the array, the `compute()` method needs to be called on the array:

In [ ]:
l3_path = Path("/luna4/maye/l3_data/")
fpaths = list(l3_path.glob("*.tif"))
fpaths[:3]

[Path('/luna4/maye/l3_data/dgdr_st_clc_cyl_20120405n_128_jp2.tif'),
 Path('/luna4/maye/l3_data/dgdr_st_clc_cyl_20100523n_128_jp2.tif'),
 Path('/luna4/maye/l3_data/dgdr_st_clc_cyl_20090705n_128_jp2.tif')]

In [ ]:
da = raster_to_xarray(fpaths[0])
da

<xarray.DataArray (band: 1, y: 20480, x: 46080)>
dask.array<open_rasterio-e88ba46e5bcbeeff5b8166d3455e3875<this-array>, shape=(1, 20480, 46080), dtype=int16, chunksize=(1, 2048, 4096), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 80.0 79.99 79.98 79.97 ... -79.97 -79.98 -79.99 -80.0
  * x        (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0 180.0
Attributes:
    transform:               (0.0078125, 0.0, -180.0, 0.0, -0.0078125, 80.0)
    crs:                     +proj=longlat +R=1737400 +no_defs=True
    res:                     (0.0078125, 0.0078125)
    is_tiled:                1
    nodatavals:              (-32768.0,)
    scales:                  (1.0,)
    offsets:                 (0.0,)
    AREA_OR_POINT:           Area
    OVR_RESAMPLING_ALG:      NEAREST
    TIFFTAG_RESOLUTIONUNIT:  3 (pixels/cm)
    TIFFTAG_XRESOLUTION:     0.39370701
    TIFFTAG_YRESOLUTION:     0.39370701

In [ ]:
da.sel(y=slice(79, 78)).max().compute()

<xarray.DataArray ()>
array(6013, dtype=int16)

In [ ]:
# export
def read_images_into_stacked_array(image_paths, name, chunk_scale=2):
    arrays = [raster_to_xarray(p, chunk_scale) for p in image_paths]
    stack = xr.concat(arrays, "time")
    stack.name = name
    return stack

In [ ]:
read_images_into_stacked_array(fpaths, "soil_temp")

,Array,Chunk
Bytes,181.05 GiB,16.00 MiB
Shape,"(103, 1, 20480, 46080)","(1, 1, 2048, 4096)"
Count,37183 Tasks,12360 Chunks
Type,int16,numpy.ndarray


In [ ]:
# export
def mapcycle_to_xarray(fpath, chunk_scale=2):
    """Read an Diviner map cycle rasterio image into a dask.array.

    The cycle identifier will be read from the filename and added
    to the xarray as a coordinate value.

    fpath: str, pathlib.Path
    chunk_scale: int
        Scaling the chunk
    """
    fpath = Path(fpath)
    da = raster_to_xarray(fpath)
    cycle = int(fpath.name.split("_")[4][:-1])
    da = da.assign_coords(band=[cycle])
    da = da.rename({"band": "mapcycle", "y": "lat", "x": "lon"})
    da.lat.attrs["long_name"] = "Latitude"
    da.lon.attrs["long_name"] = "Longitude"
    for coord in ["lat", "lon"]:
        da[coord].attrs["units"] = "deg"
    return da

In [ ]:
mapcycle_to_xarray(fpaths[0])

<xarray.DataArray (mapcycle: 1, lat: 20480, lon: 46080)>
dask.array<open_rasterio-971342462e4be0714045001bd98e366c<this-array>, shape=(1, 20480, 46080), dtype=int16, chunksize=(1, 2048, 4096), chunktype=numpy.ndarray>
Coordinates:
  * mapcycle  (mapcycle) int64 20120405
  * lat       (lat) float64 80.0 79.99 79.98 79.97 ... -79.98 -79.99 -80.0
  * lon       (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
Attributes:
    transform:               (0.0078125, 0.0, -180.0, 0.0, -0.0078125, 80.0)
    crs:                     +proj=longlat +R=1737400 +no_defs=True
    res:                     (0.0078125, 0.0078125)
    is_tiled:                1
    nodatavals:              (-32768.0,)
    scales:                  (1.0,)
    offsets:                 (0.0,)
    AREA_OR_POINT:           Area
    OVR_RESAMPLING_ALG:      NEAREST
    TIFFTAG_RESOLUTIONUNIT:  3 (pixels/cm)
    TIFFTAG_XRESOLUTION:     0.39370701
    TIFFTAG_YRESOLUTION:     0.39370701

In [ ]:
# export
def read_mapcycles_into_stacked_array(image_paths, name, chunk_scale=2):
    with xr.set_options(keep_attrs=True):
        arrays = [mapcycle_to_xarray(p, chunk_scale) for p in image_paths]
        stack = xr.concat(arrays, "mapcycle")
        stack.name = name
        return stack

In [ ]:
read_mapcycles_into_stacked_array(fpaths, "soil_temp")

,Array,Chunk
Bytes,181.05 GiB,16.00 MiB
Shape,"(103, 20480, 46080)","(1, 2048, 4096)"
Count,24823 Tasks,12360 Chunks
Type,int16,numpy.ndarray
